In [6]:
import pandas as pd
from tqdm.notebook import tqdm
import glob

In [2]:
def load_test():    
    dfs = []
    for e, chunk_file in enumerate(tqdm(glob.glob('../../data/train_data/test_parquet/*'))):
        chunk = pd.read_parquet(chunk_file)
        #chunk.ts *= 1000
        dfs.append(chunk)

    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [7]:
test_df = load_test()

  0%|          | 0/17 [00:00<?, ?it/s]

In [8]:
%%time
test_df = test_df.sort_values(["session", "ts"])
test_df['d'] = test_df.groupby('session').ts.diff()
test_df.d = (test_df.d > 60*60*2).astype('int16').fillna(0)
test_df.d = test_df.groupby('session').d.cumsum()

CPU times: user 4min 19s, sys: 7.62 s, total: 4min 26s
Wall time: 4min 20s


In [12]:
test_df.dtypes

session    int32
aid        int32
ts         int32
type        int8
d          int16
dtype: object

In [11]:
test_df.session = test_df.session.astype('int32')
test_df.aid = test_df.aid.astype('int32')

In [13]:
test_df.to_parquet('test_with_d.parquet',index=False)